In [7]:
#Install following libraries if not installed already
#!pip install sodapy
#!pip install plotly

In [8]:
#Import libraries
from sodapy import Socrata
import plotly.offline as pyo
import plotly.graph_objs as go
import ipywidgets as widgets
import pandas as pd
import numpy as np
from scipy import special

#py.offline.init_notebook_mode(connected=True)

In [9]:
client = Socrata("data.sfgov.org", None)

In [10]:
results = client.get("wg3w-h783", limit=2000)

In [11]:
results_df = pd.DataFrame.from_records(results)

In [12]:
results_df.columns

Index([':@computed_region_26cr_cadq', ':@computed_region_2dwj_jsy4',
       ':@computed_region_6pnf_4xz7', ':@computed_region_6qbp_sg9q',
       ':@computed_region_ajp5_b2md', ':@computed_region_h4ep_8xdi',
       ':@computed_region_jg9y_a9du', ':@computed_region_nqbw_i6c3',
       ':@computed_region_qgnn_b9vv', ':@computed_region_y6ts_4iup',
       'analysis_neighborhood', 'cad_number', 'cnn', 'filed_online',
       'incident_category', 'incident_code', 'incident_date',
       'incident_datetime', 'incident_day_of_week', 'incident_description',
       'incident_id', 'incident_number', 'incident_subcategory',
       'incident_time', 'incident_year', 'intersection', 'latitude',
       'longitude', 'point', 'police_district', 'report_datetime',
       'report_type_code', 'report_type_description', 'resolution', 'row_id',
       'supervisor_district'],
      dtype='object')

In [13]:
#Taking a copy of original dataframe
results_df_copy = results_df.copy()

In [14]:
#Dropping junk columns
results_df_copy = results_df_copy[results_df_copy.columns.drop(list(results_df_copy.filter(regex=':@computed_region')))]

In [16]:
results_df_copy.columns

Index(['analysis_neighborhood', 'cad_number', 'cnn', 'filed_online',
       'incident_category', 'incident_code', 'incident_date',
       'incident_datetime', 'incident_day_of_week', 'incident_description',
       'incident_id', 'incident_number', 'incident_subcategory',
       'incident_time', 'incident_year', 'intersection', 'latitude',
       'longitude', 'point', 'police_district', 'report_datetime',
       'report_type_code', 'report_type_description', 'resolution', 'row_id',
       'supervisor_district'],
      dtype='object')

In [46]:
results_groupby = results_df_copy.groupby(by='incident_day_of_week', as_index=False)['incident_description'].count()
results_groupby = pd.DataFrame(results_groupby)
results_groupby

#list(results_groupby['incident_day_of_week'])

,incident_day_of_week,incident_description
0,Friday,208
1,Monday,417
2,Saturday,212
3,Sunday,264
4,Thursday,191
5,Tuesday,389
6,Wednesday,319


In [53]:
data = [go.Scatter(
    x = results_groupby['incident_day_of_week'],
    y = results_groupby['incident_description'],
    mode = 'markers',
    marker = dict(
        size = 14,
        color = 'rgb(51,204,153)',
        symbol = 'pentagon',
        line = dict(
            width = 2)
    )
    )]

layout = go.Layout(
    title = 'Number of Incidents by Day of Week',
    xaxis = dict(title='Incident Day of Week'),
    yaxis = dict(title='Number of Incidents'),
    hovermode='closest'
)

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='scatter.html')

'file:///Users/bharath/Desktop/Apps/Coursera/Github/SanFrancisco_Crime/scatter.html'

SF_Crime.ipynb scatter.html
